# JSN_XGBoost_machine
Training with numerical data JSN prediction


author = MV<br>
date = 2021-11-04<br>

_______________________________________

description

# Imports

In [ ]:
! nvidia-smi

In [ ]:
# this defines the GPU you are using
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
# add paths for dnn2 and labelbox-connector
import sys
sys.path.insert(1, "/srv/dnn-framework")
sys.path.insert(1, "/srv/labelbox-connector")

In [ ]:
#general
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import logging
import cv2
import copy

#tensorflow
import tensorflow as tf
from tensorflow import keras

In [ ]:
# dnn2 framework
from framework.dnn_app.dnn_app import DnnApp
from framework.data_objects import *
from framework.dataset.dataset_collection import DatasetCollection
from framework.dataset.csv_data_helper import CsvDataHelper
from framework.batch_generator.batch_generator import BatchGenerator
from framework.models.model_generator import ModelGenerator
from framework.trainings_scheduler.trainings_scheduler import TrainingsScheduler

# dnn2 networks
from networks.models.ModelTypes import ModelTypes

In [ ]:
# Import libraries that will be needed for the lab

import xgboost as xgb
import numpy as np
from collections import OrderedDict
import gc
from glob import glob
import os
import pandas as pd
from copy import copy
from time import time
from sklearn.metrics import roc_auc_score,confusion_matrix,accuracy_score,classification_report,roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from timeit import default_timer
import matplotlib.pyplot as plt
import pickle



# Set the seed for numpy
np.random.seed(123)

#general
import matplotlib.pylab as plt
import matplotlib
import numpy as np
import pandas as pd
import os
import logging
import cv2
import albumentations as A

#dnn
import keras.backend as K
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard, EarlyStopping
from keras.optimizers import Adam
from keras.layers import Conv2D
from keras.models import Model, load_model

#framework
from framework.utils.app_base import DnnApp
from framework.utils import Landmarks, BoundingBox, img_utils
from framework.loader import LmBlobBatchGenerator
from framework.dataset import DatasetAugmented
from framework.preprocessing import preprocessing, image_processing
from framework.augmentor import AugmentorImage

from imgaug import augmenters as iaa
from sklearn.metrics import confusion_matrix, mean_squared_error, cohen_kappa_score, roc_auc_score, roc_curve, log_loss
from sklearn.metrics import precision_recall_fscore_support, classification_report
import itertools

from datetime import date

import keras
from keras.applications.resnet50 import ResNet50
from keras.models import Model

from network.iblmodel import IBLModel as Model
from network.RetinaNet.resnet import ResNet2D50
from keras.layers import Input, Dense, LeakyReLU

In [ ]:
#configs
%matplotlib inline
logging.basicConfig(format='%(asc' 'time)s %(name)-25s %(level' 'name)-8s %(message)s')
logging.getLogger().setLevel(logging.INFO) # you change this to logging.DEBUG to get more logging information

In [ ]:
#session config - if needed
import tensorflow.keras.backend as K
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
K.set_session(tf.Session(config=config))

# Code

## Normalize data

# Add knee injection to JSN20

In [ ]:
mod = pd.read_csv("/srv/XGBoost/Master_dataset_JSN.csv")
#mod = mod[['Knee_inj', 'Laterality','ID']]
mod.columns

In [ ]:
test = df.merge(mod, on =['Laterality', 'ID'], how = 'left')

In [ ]:
from sklearn import preprocessing

test = test.replace({'Knee_inj': {'No':0, 'Yes':1, '1: Yes': 1, '0: No':0}})

## StandardScaler
col_names = ['Knee_inj']
new_col_names = ['Knee_inj_S']
features = test[col_names]

scaler = preprocessing.StandardScaler().fit(features.values)
features = scaler.transform(features.values)
scaled_features = pd.DataFrame(features, columns = new_col_names)
scaled_features['ID'] = test.ID
scaled_features['Laterality'] = test.Laterality

scaled_features = scaled_features.dropna()


In [ ]:
scaled_features

# Load Dataset

In [ ]:
data_path = '/srv/XGBoost/Master_10JSN_XML_ex014.csv'
df =  pd.read_csv(data_path)
df.columns

In [ ]:
df = df.merge(scaled_features, on = ['Laterality','ID'], how = 'left')

In [ ]:
## drop all undesired columns
df = df.drop(columns={'KL', 'Implant','dicom_img_path.1'})
print(len(df))
df = df.dropna()
print(len(df))

In [ ]:
pd.DataFrame(df['class'].value_counts())

In [ ]:
# here we train a label encoder so that we can map our classes to integers later for model training
le = LabelEncoder()
le.fit(df['class'])
print(le.classes_)

## Split into Train and test set

In [ ]:
## For standard scaler data
df_num = df[[
    "Patient Age_S",
    'BMI_S',
    "Patient Sex_S",
    "Hip_symptoms_S",
    "Knee_inj_S",
    "WOMAC_dis_S",
    "WOMAC_pain_S",
    "WOMAC_stiff_S",
    "sclerosis_S",
    "osteophytes_S",
    "KL-grade_S",
    'other_knee_KOA_S',
    'class']]

In [ ]:
## For original data
df_num = df[[
    "Patient Age",
    'BMI',
    "Patient Sex",
    "Hip_symptoms",
    'Knee_inj',
    "WOMAC_dis",
    "WOMAC_pain",
    "WOMAC_stiff",
    "KL-grade_a",
    "sclerosis_a",
    "osteophytes_a",
    'other_knee_KOA',
    'class']]

In [ ]:
# capture the labels
labels = df['class'].copy()

# split data into test and train
x_train, x_test, y_train, y_test = train_test_split(df_num,
                                                    labels,
                                                    test_size=.25, 
                                                    random_state=42)

In [ ]:
# check that the dimensions of our train and test sets are okay
print(x_train.shape)
print('Number of slow progressors in Train set:',len(x_train.loc[x_train['class'] == 0]))
print('Number of fast progressors in Train set:',len(x_train.loc[x_train['class'] == 1]))

print(y_train.shape)
print(x_test.shape)
print('Number of slow progressors in Test set:',len(x_test.loc[x_test['class'] == 0]))
print('Number of slow progressors in Test set:',len(x_test.loc[x_test['class'] == 1]))
print(y_test.shape)

# Training Binary classifier

In [ ]:
x_train = x_train.drop(columns={'class'})
x_test = x_test.drop(columns={'class'})


In [ ]:
## print most important features 

from xgboost import XGBRegressor
xgbr = XGBRegressor(n_estimators=100)
xgbr.fit(x_train, y_train)

plt.barh(x_train.columns, xgbr.feature_importances_)
#plt.show()
print(x_train.columns)
print(xgbr.feature_importances_)

In [ ]:
# Grid search for hyperparameters
from sklearn.metrics import roc_curve, auc
tx = 0.4
for lr in {0.1,0.2,0.4}:
    for max_depth in {8, 32, 64, 128}:
        for alpha in {0.1,0.2,0.5, 0.9}:
            for gamma in {0.2,0.5,0.7,0.9}:
        
                for max_leaves in {4,8, 16}:
                    params = {
                        'num_rounds':        50,
                        'max_depth':         max_depth,
                        'max_leaves':        2**max_leaves,
                        'alpha':             alpha,
                        'eta':               0.1,
                        'gamma':             gamma,
                        'learning_rate':     lr,
                        'subsample':         1,
                        'reg_lambda':        1,
                        'scale_pos_weight':  2,
                        'tree_method':       'gpu_hist',
                        'objective':         'binary:logistic',
                        'verbose':           True
                    }

                    dtrain = xgb.DMatrix(x_train, label=y_train)
                    dtest = xgb.DMatrix(x_test, label=y_test)
                    evals = [(dtest, 'test',), (dtrain, 'train')]

                    num_rounds = params['num_rounds']

                    model = xgb.train(params, dtrain, num_rounds, evals=evals)

                    # evaluation
                    threshold = .5
                    true_labels = y_test.astype(int)
                    preds = model.predict(dtest)
                    fpr, tpr, t = roc_curve(true_labels, preds)
                    au = auc(fpr,tpr)

                    if au > tx:
                        tx = au
                        best_lr = lr
                        best_depth = max_depth
                        best_leaves = max_leaves
                        
                        best_alpha = alpha
                        best_gamma = gamma
                        print(tx)

print(tx, best_lr, best_depth, best_leaves, best_alpha, best_gamma)

In [ ]:
params = {
    'num_rounds':        75,
    'max_depth':         8,
    'max_leaves':        2**4,
    'alpha':             0.1,
    'eta':               0.1,
    'gamma':             0.9,
    'learning_rate':     0.1,
    'subsample':         1,
    'reg_lambda':        1,
    'scale_pos_weight':  2,
    'tree_method':       'gpu_hist',
    'objective':         'binary:logistic',
    'verbose':           True
}

In [ ]:
%%time 

dtrain = xgb.DMatrix(x_train, label=y_train)
dtest = xgb.DMatrix(x_test, label=y_test)
evals = [(dtest, 'test',), (dtrain, 'train')]

In [ ]:
num_rounds = params['num_rounds']

In [ ]:
model = xgb.train(params, dtrain, num_rounds, evals=evals)

# Testing and Evalution 

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
threshold = .47
true_labels = y_test.astype(int)
true_labels.sum()

In [ ]:
preds = model.predict(dtest)

pred_labels = (preds > threshold).astype(int)
print(preds)

In [ ]:
pred_labels.sum()

In [ ]:
fpr, tpr, t = roc_curve(true_labels, preds)
auc(fpr,tpr)

In [ ]:
results = confusion_matrix(true_labels, pred_labels) 

def plot_confusion_matrix(cm, target_names, title='Confusion Matrix', cmap=plt.cm.Greens):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation=45)
    plt.yticks(tick_marks, target_names)
    plt.tight_layout()

    width, height = cm.shape

    for x in range(width):
        for y in range(height):
            plt.annotate(str(cm[x][y]), xy=(y, x), 
                        horizontalalignment='center',
                        verticalalignment='center')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')


plot_confusion_matrix(results, ['slow','fast'])

In [ ]:
plt.ylim((0,1))
plt.xlim((0,1))
plt.plot(fpr,tpr)
plt.plot([0,1],[0,1])
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.plot(fpr,t)